In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import GRU, Dense, Input # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
import matplotlib.pyplot as plt

# Sample data: sequence of numbers
data = np.sin(np.linspace(0, 100, 1000))

# Normalize the data
data = (data - np.min(data)) / (np.max(data) - np.min(data))

# Function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define sequence length
seq_length = 10

# Create sequences
X, y = create_sequences(data, seq_length)

df = pd.DataFrame(X, columns=[f'f_t-{seq_length-i}' for i in range(seq_length)])
df['f_t'] = y
df.round(2).head()


,f_t-10,f_t-9,f_t-8,f_t-7,f_t-6,f_t-5,f_t-4,f_t-3,f_t-2,f_t-1,f_t
0,0.50,0.55,0.60,0.65,0.69,0.74,0.78,0.82,0.86,0.89,0.92
1,0.55,0.60,0.65,0.69,0.74,0.78,0.82,0.86,0.89,0.92,0.95
2,0.60,0.65,0.69,0.74,0.78,0.82,0.86,0.89,0.92,0.95,0.97
3,0.65,0.69,0.74,0.78,0.82,0.86,0.89,0.92,0.95,0.97,0.98
4,0.69,0.74,0.78,0.82,0.86,0.89,0.92,0.95,0.97,0.98,0.99


In [26]:
import numpy as np
from keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import GRU, Dense, Input # type: ignore
import pandas as pd

# Generate Fibonacci sequence
def generate_fibonacci(n):
    fib_sequence = [0, 1]
    for i in range(2, n):
        next_value = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_value)
    return fib_sequence

# Prepare the dataset
def create_dataset(sequence):
    X, y = [], []
    for i in range(len(sequence) - 2):
        X.append([sequence[i], sequence[i+1]])
        y.append(sequence[i+2])
    return np.array(X), np.array(y)

# Parameters
n_seq = 20  # Length of the Fibonacci sequence

# Generate the Fibonacci sequence
fib_sequence = generate_fibonacci(n_seq + 2)  # Generate enough values for input-output pairs

# Create the dataset
X, y = create_dataset(fib_sequence)

seq_length = 2
df = pd.DataFrame(X, columns=[f'f_t-{seq_length-i}' for i in range(seq_length)])
df['f_t'] = y

In [34]:
# Reshape X to be [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the GRU model
model = Sequential()
model.add(Input(shape=(2, 1)))
model.add(GRU(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train the model
history = model.fit(X, y, epochs=100, validation_data=(X_test, y_test), verbose = 0)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test, verbose =0)
f_hat = model.predict(X, verbose =0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 3217.7212
Test Loss: 3217.72119140625


In [35]:
df['f_pred'] = f_hat
df.round(2)

,f_t-2,f_t-1,f_t,f_pred
0,0,1,1,1.590000
1,1,1,2,2.480000
2,1,2,3,3.080000
3,2,3,5,4.820000
4,3,5,8,7.450000
5,5,8,13,12.650000
6,8,13,21,21.570000
7,13,21,34,36.779999
8,21,34,55,60.599998
9,34,55,89,96.889999
